In [2]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
import os
import sys
import urllib.request

In [24]:
df_relmovie = pd.read_csv('relmovie.csv',skiprows=2)
df_actor = pd.read_csv('actor.csv',index_col=0)

In [ ]:
df_relmovie.head(100)

In [26]:
df_relmovie1=df_relmovie.drop(['제작사','수입사','영화유형','영화형태','서울관객수','서울매출액','영화구분'],axis=1)

In [27]:
df_relmovie1.head(1)

순번 영화명   감독        배급사         개봉일  국적  전국스크린수         전국매출액        전국관객수  \
0  1   명량  김한민  씨제이이앤엠(주)  2014-07-30  한국  1,587   135748398910  17,613,682    

   장르        등급  
0  사극  15세이상관람가

In [28]:
df_relmovie2 = df_relmovie1.rename(columns={'순번':'rank','영화명':'mname','감독':'director','배급사':'provider','개봉일':'date','국적':'country','전국스크린수':'n_scrn','전국매출액':'money','전국관객수':'guest','장르':'genre','등급':'rating'}, inplace=False)

In [29]:
df_relmovie2.head(4)

rank      mname director       provider        date country  n_scrn  \
0   1          명량      김한민      씨제이이앤엠(주)  2014-07-30      한국  1,587    
1   2   신과함께-죄와 벌      김용화  롯데쇼핑㈜롯데엔터테인먼트  2017-12-20      한국  1,912    
2   3        국제시장      윤제균      씨제이이앤엠(주)  2014-12-17      한국    966    
3   4         베테랑      류승완      씨제이이앤엠(주)  2015-08-05      한국  1,064    

          money        guest genre    rating  
0  135748398910  17,613,682     사극  15세이상관람가  
1  115699634137  14,410,931    판타지  12세이상관람가  
2  110828014630  14,245,998    드라마  12세이상관람가  
3  105024756250  13,395,400     액션  15세이상관람가

In [30]:
cnt=0
for i in df_relmovie2['n_scrn']:
    i = float(str(i).replace(',',''))
    df_relmovie2.set_value(cnt, "n_scrn", i)
    cnt=cnt+1

C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [31]:
cnt=0
for i in df_relmovie2['guest']:
    i = int(str(i).replace(',',''))
    df_relmovie2.set_value(cnt, "guest", i)
    cnt=cnt+1

C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [32]:
df_relmovie2.head(1)

rank mname director   provider        date country n_scrn         money  \
0   1     명량      김한민  씨제이이앤엠(주)  2014-07-30      한국   1587  135748398910   

      guest genre    rating  
0  17613682    사극  15세이상관람가

In [54]:
df_relmovie2['grade'] = ''
df_relmovie2['actor'] = ''

In [51]:
df_relmovie2.head(1)

rank mname director   provider        date country n_scrn         money  \
0   1     명량      김한민  씨제이이앤엠(주)  2014-07-30      한국   1587  135748398910   

      guest genre    rating grade actor  
0  17613682    사극  15세이상관람가

In [ ]:
encText = urllib.parse.quote('아이언맨3')
url = "https://openapi.naver.com/v1/search/movie.xml?yearfrom=2011&yearto=2018&display=100&query=" + encText # xml 결과

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read().decode('utf-8')
    soup2 = BeautifulSoup(response_body,"html.parser")
    print(soup2.prettify())

In [96]:
for it_title in soup2.find_all('item'):
    print((it_title.title).text.lstrip('<br>').rstrip('</br>'))

검사외전


In [111]:
'안소니 루소|조 루소|'.replace("|",",").rstrip(',').strip()

'안소니 루소,조 루소'

In [114]:
client_id = "D4rJqpB5sQrGOQZ098Xk"
client_secret = "PhSvV07Pho"

userrating=[]
actor=[]
director=[]
title=[]

keyword=df_relmovie2['mname']
cnt = 0
for name_cnt in range(len(keyword)):
    encText = urllib.parse.quote(keyword[name_cnt])
    url = "https://openapi.naver.com/v1/search/movie.xml?yearfrom=2011&yearto=2018&display=100&query=" + encText # xml 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read().decode('utf-8')
        soup2 = BeautifulSoup(response_body,"html.parser")
        sleep(0.05)        
        for dir_cnt in soup2.find_all('director'):
            director.append(dir_cnt.text.replace("|",",").rstrip(',').strip())
        for it_title in soup2.find_all('item'):
            title.append(it_title.title.text.lstrip('<br>').rstrip('</br>'))
            
        for usr_rat in soup2.findAll('userrating'): 
            userrating.append(usr_rat.text) 
        for act_name in soup2.findAll('actor'):
            actor.append(act_name.text)

        for dir_cor in range(len(director)):
            if (director[dir_cor] == df_relmovie2[keyword==keyword[name_cnt]]['director'].values[0].strip())  & \
               (title[dir_cor] == df_relmovie2[keyword==keyword[name_cnt]]['mname'].values[0].strip()) :
#             if keyword[name_cnt] == director[dir_cor]:
                df_relmovie2.set_value(name_cnt,'grade',userrating[dir_cor])
                df_relmovie2.set_value(name_cnt,'actor',actor[dir_cor])
    print(name_cnt)
print("-----------complete------------")

C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

AttributeError: 'float' object has no attribute 'strip'

In [117]:
df_relmovie2.head(100)

rank               mname       director                  provider  \
0     1                   명량            김한민                 씨제이이앤엠(주)   
1     2            신과함께-죄와 벌            김용화             롯데쇼핑㈜롯데엔터테인먼트   
2     3                 국제시장            윤제균                 씨제이이앤엠(주)   
3     4                  베테랑            류승완                 씨제이이앤엠(주)   
4     5                  도둑들            최동훈                    (주)쇼박스   
5     6              7번방의 선물            이환경       (주)넥스트엔터테인먼트월드(NEW)   
6     7                   암살            최동훈                    (주)쇼박스   
7     8          광해, 왕이 된 남자            추창민                 씨제이이앤엠(주)   
8     9                택시운전사             장훈                    (주)쇼박스   
9    10                  부산행            연상호       (주)넥스트엔터테인먼트월드(NEW)   
10   11                  변호인            양우석       (주)넥스트엔터테인먼트월드(NEW)   
11   12         어벤져스: 인피니티 워    안소니 루소,조 루소        월트디즈니컴퍼니코리아 유한책임회사   
12   13     어벤져스: 에이지 오브 울트론          조스 웨던            월트디즈니컴퍼니코리아(주)   
13   14                 겨울왕국    크리스 벅,제니퍼 리  소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)   
14   15                인터스텔라       크리스토퍼 놀란             워너브러더스 코리아(주)   
15   16                 검사외전            이일형                    (주)쇼박스   
16   17                   관상            한재림                    (주)쇼박스   
17   18               아이언맨 3          쉐인 블랙  소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)   
18   19                 설국열차            봉준호                 씨제이이앤엠(주)   
19   20        캡틴 아메리카: 시빌 워    안소니 루소,조 루소            월트디즈니컴퍼니코리아(주)   
20   21               수상한 그녀            황동혁                 씨제이이앤엠(주)   
21   22         해적: 바다로 간 산적            이석훈             롯데쇼핑㈜롯데엔터테인먼트   
22   23                   공조            김성훈                 씨제이이앤엠(주)   
23   24              트랜스포머 3         마이클 베이                 씨제이이앤엠(주)   
24   25                 히말라야            이석훈                 씨제이이앤엠(주)   
25   26       미션임파서블:고스트프로토콜         브래드 버드                 씨제이이앤엠(주)   
26   27                   밀정            김지운             워너브러더스 코리아(주)   
27   28               최종병기 활            김한민             롯데쇼핑㈜롯데엔터테인먼트   
28   29                   써니            강형철                 씨제이이앤엠(주)   
29   30          스파이더맨: 홈 커밍           존 와츠  소니픽쳐스엔터테인먼트코리아주식회사극장배급지점   
..   ...                 ...            ...                       ...   
70   71                   마션         리들리 스콧              이십세기폭스코리아(주)   
71   72            토르: 라그나로크       타이카 와이티티        월트디즈니컴퍼니코리아 유한책임회사   
72   73           어메이징 스파이더맨           마크 웹       한국소니픽쳐스릴리징브에나비스타영화㈜   
73   74      님아, 그 강을 건너지 마오            진모영        CGV아트하우스,(주)대명문화공장   
74   75    조선명탐정 : 각시투구꽃의 비밀            김석윤                    (주)쇼박스   
75   76           군도: 민란의 시대            윤종빈                    (주)쇼박스   
76   77   범죄와의 전쟁: 나쁜놈들 전성시대            윤종빈                    (주)쇼박스   
77   78                 주토피아  바이론 하워드,리치 무어            월트디즈니컴퍼니코리아(주)   
78   79           엣지 오브 투모로우         더그 라이만             워너브러더스 코리아(주)   
79   80                  신세계            박훈정       (주)넥스트엔터테인먼트월드(NEW)   
80   81             신비한 동물사전        데이빗 예이츠             워너브러더스 코리아(주)   
81   82                  도가니            황동혁                 씨제이이앤엠(주)   
82   83           내 아내의 모든 것            민규동       (주)넥스트엔터테인먼트월드(NEW)   
83   84                  판도라            박정우       (주)넥스트엔터테인먼트월드(NEW)   
84   85                  연가시            박정우                 씨제이이앤엠(주)   
85   86                  강철비            양우석       (주)넥스트엔터테인먼트월드(NEW)   
86   87        해리포터와 죽음의 성물2        데이빗 예이츠             워너브러더스 코리아(주)   
87   88   엑스맨: 데이즈 오브 퓨처 패스트        브라이언 싱어              이십세기폭스코리아(주)   
88   89                  아가씨            박찬욱                 씨제이이앤엠(주)   
89   90         어메이징 스파이더맨 2           마크 웹  소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)   
90   91                  용의자            원신연                    (주)쇼박스   
91   92                건축학개론            이용주             롯데쇼핑㈜롯데엔터테인먼트   
92   93                  댄싱퀸            이석훈                 씨제

In [118]:
df_relmovie2.to_excel('jini.xlsx',sheet_name='Sheet1')